<a href="https://colab.research.google.com/github/pbhatt48/python_bootcamp/blob/master/MLNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regular libraries:
```
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
```
Read CSV and Drop:

```
df_sd = pd.read_csv(survey_data,  na_values='NA')
df_sd = df_sd.dropna()
```
Some Exploratory:
```
USAhousing.info()
USAhousing.describe()
USAhousing.columns
sns.pairplot(USAhousing)
sns.distplot(USAhousing['Price'])
sns.heatmap(USAhousing.corr())

sns.jointplot(x='fico',y='int.rate',data=loans,color='purple')

```

Get Index and drop duplicates:

```
df_sd.index
df_su.drop_duplicates(subset ="user_id",
                     keep = 'first' , inplace = True)
```

Merget two dataframe:
```
merged_df = pd.merge(df_sd, df_su, on=["user_id"])
merged_df
```

Get columns:
```
merged_df.columns
```
Filtering:
```
merged_df[(merged_df.duolingo_subscriber == "No, but I have previously paid for Duolingo Plus") & (merged_df.purchased_subscription == False)]
```

GET X and Y:

```
X = merged_df[['user_id', 'age', 'annual_income', 'country', 'duolingo_platform',
        'duolingo_usage', 'employment_status',
       'future_contact', 'gender', 'longest_streak',
       'n_days_on_platform' ]]

y = merged_df[['duolingo_subscriber' ]]
```

Get Unique Items:

```
X.age.nunique
```

Replace True False to 0 and 1


```
final_df['purchased_subscription'] = final_df['purchased_subscription'].replace([True],1)
```

Encoding:


```
age = pd.get_dummies(X.age, drop_first=True)
annual_income = pd.get_dummies(X.annual_income, drop_first=True)

final_df.drop(['age', 'annual_income', 'country', 'duolingo_platform',
        'duolingo_usage' ], axis=1, inplace=True)
final_df = pd.concat([final_df, age, annual_income, country,duolingo_platform, duolingo_usage, employment_status, 
                      future_contact, gender, primary_language_commitment,primary_language_review,
                      primary_language_motivation, primary_language_motivation_followup,
                      primary_language_proficiency, student, survey_complete, took_placement_test],axis=1)
final_df.head(10)
```

Balance dataset:
```
shuffled_df = final_df.sample(frac=1,random_state=42)

# Put all the fraud class in a separate dataset.
class1 = shuffled_df.loc[shuffled_df['purchased_subscription'] == 1]
class0 = shuffled_df.loc[shuffled_df['purchased_subscription'] == 0].sample(n=410,random_state=42)

# Concatenate both dataframes again
balanced_df = pd.concat([class1, class0])

#plot the dataset after the undersampling
plt.figure(figsize=(8, 8))
sns.countplot('purchased_subscription', data=balanced_df)
plt.title('Balanced Classes')
plt.show()

```

Drop a column:
```
X = balanced_df.drop('purchased_subscription',axis=1)
y = balanced_df['purchased_subscription']
```
Scale features or Normalize
```
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_features = scaler.transform(X)
df_feat = pd.DataFrame(scaled_features,columns=X.columns)
df_feat.head()
X= df_feat
```

Split Data
```
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.25,random_state=42)
```

Linear Regression:
```
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

Model Evaluation:
# print the intercept
print(lm.intercept_)
coeff_df = pd.DataFrame(lm.coef_,X.columns,columns=['Coefficient'])
coeff_df

Prediction:
predictions = lm.predict(X_test)
plt.scatter(y_test,predictions)
sns.distplot((y_test-predictions),bins=50);

Metrics:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
```

Logistic Regression:
```
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

Prediction:

predictions = logmodel.predict(X_test)

Metrics:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))
```

KNN LEARNER

```
sns.pairplot(df,hue='TARGET CLASS',palette='coolwarm')

For KNN you will need to scale the values
scaler = StandardScaler()
scaler.fit(df.drop('TARGET CLASS',axis=1))
scaled_features = scaler.transform(df.drop('TARGET CLASS',axis=1))
df_feat = pd.DataFrame(scaled_features,columns=df.columns[:-1])

Train test::
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)


Predictions::
pred = knn.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))


Choosing K value::

error_rate = []

# Will take some time
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))


Create PLOT:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')


Retrain with NEW optimal K value:
# NOW WITH K=30
knn = KNeighborsClassifier(n_neighbors=30)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=30')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

```

DECISION TREES AND RANDOM FOREST::

```
SPLIT::

from sklearn.model_selection import train_test_split
X = final_data.drop('not.fully.paid',axis=1)
y = final_data['not.fully.paid']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

DTREE:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

PREDICTIONS:
predictions = dtree.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

RANDOM FOREST::
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=600)
rfc.fit(X_train,y_train)

PREDICTIONS:
predictions = rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

```

SVC:

```
from sklearn.svm import SVC
model = SVC()
model.fit(X_train,y_train)

predictions = model.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

GRIDSEARCH::

param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']}

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
grid.fit(X_train,y_train)

GRID PARAMS:

grid.best_params_
grid.best_estimator_
grid_predictions = grid.predict(X_test)
print(classification_report(y_test,grid_predictions))


```

KMEANS CLUSTERING:

```
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
kmeans.fit(data[0])
kmeans.cluster_centers_
kmeans.labels_

#Clustering plots:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(10,6))
ax1.set_title('K Means')
ax1.scatter(data[0][:,0],data[0][:,1],c=kmeans.labels_,cmap='rainbow')
ax2.set_title("Original")
ax2.scatter(data[0][:,0],data[0][:,1],c=data[1],cmap='rainbow')



ELBOW CURVE::
#import matplotlib.style as style style.available
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#plt.style.use('fivethirtyeight')
sse = []
list_k = list(range(1, 50, 5))

for k in list_k:
    km = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42, max_iter=400)
    km.fit(X_train)
    sse.append(km.inertia_)

# Plot sse against k
#plt.figure(figsize=(4, 4))
plt.plot(list_k, sse, '-o', linewidth=2, label="Inertia")
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title("Elbow Curve(PBM)")
#plt.xlim(0, list_k[-1])
plt.xticks(list_k)
plt.legend()
```